In [1]:
from pyhealth.datasets import MIMIC4Dataset
if __name__ == "__main__":
    dataset = MIMIC4Dataset(
        ehr_root="/srv/local/data/physionet.org/files/mimiciv/2.2/",
        ehr_tables=["diagnoses_icd", "procedures_icd", "labevents", "prescriptions"],
        dev=True,
        cache_dir="../benchmark_cache/mimic4_ehr_with_prescr/"
    )

    from pyhealth.tasks import InHospitalMortalityMIMIC4

    task = InHospitalMortalityMIMIC4()
    samples = dataset.set_task(task, num_workers=4, cache_dir="../benchmark_cache/mimic4_ihm_w_pre2/")

    from pyhealth.datasets import split_by_sample


    train_dataset, val_dataset, test_dataset = split_by_sample(
        dataset=samples, ratios=[0.7, 0.1, 0.2]
    )




Memory usage Starting MIMIC4Dataset init: 772.4 MB
Initializing mimic4 dataset from /srv/local/data/physionet.org/files/mimiciv/2.2/|None|None (dev mode: True)
Initializing MIMIC4EHRDataset with tables: ['diagnoses_icd', 'procedures_icd', 'labevents', 'prescriptions'] (dev mode: True)
Using default EHR config: /home/johnwu3/projects/PyHealth_Branch_Testing/PyHealth/pyhealth/datasets/configs/mimic4_ehr.yaml
Memory usage Before initializing mimic4_ehr: 772.4 MB
Initializing mimic4_ehr dataset from /srv/local/data/physionet.org/files/mimiciv/2.2/ (dev mode: True)
Memory usage After initializing mimic4_ehr: 772.5 MB
Memory usage After EHR dataset initialization: 772.5 MB
Memory usage Completed MIMIC4Dataset init: 772.5 MB
Setting task InHospitalMortalityMIMIC4 for mimic4 base dataset...
Combining data from ehr dataset
Scanning table: diagnoses_icd from /srv/local/data/physionet.org/files/mimiciv/2.2/hosp/diagnoses_icd.csv.gz
Joining with table: /srv/local/data/physionet.org/files/mimiciv/2

2025-12-18 20:49:34,422 - distributed.shuffle._worker_plugin - ERROR - [Errno 39] Directory not empty: PosixPath('/home/johnwu3/projects/PyHealth_Branch_Testing/PyHealth/benchmark_cache/mimic4_ehr_with_prescr/dask_scratch/dask-scratch-space/worker-89c2re6g/shuffle-8936a2b693236eda400883caa2af525d-2')
Traceback (most recent call last):
  File "/home/johnwu3/miniconda3/envs/pyhealth312/lib/python3.12/site-packages/distributed/utils.py", line 825, in wrapper
    return await func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/johnwu3/miniconda3/envs/pyhealth312/lib/python3.12/site-packages/distributed/shuffle/_worker_plugin.py", line 76, in close
    await shuffle_run.close()
  File "/home/johnwu3/miniconda3/envs/pyhealth312/lib/python3.12/site-packages/distributed/shuffle/_core.py", line 311, in close
    await self._disk_buffer.close()
  File "/home/johnwu3/miniconda3/envs/pyhealth312/lib/python3.12/site-packages/distributed/shuffle/_disk.py", line 228, in close
 

Caching event dataframe to ../benchmark_cache/mimic4_ehr_with_prescr/global_event_df.parquet...
Applying task transformations on data...


100%|██████████| 1000/1000 [00:30<00:00, 33.20it/s]


Indexing progress:   0%|          | 0/1 [00:00<?, ?step/s]


Fitting processors on the dataset...


/tmp/ipykernel_1324622/2254465948.py:13: UserWarning: A newer version of litdata is available (0.2.59). Please consider upgrading with `pip install -U litdata`. Not all functionalities of the platform can be guaranteed to work with the current version.
  samples = dataset.set_task(task, num_workers=4, cache_dir="../benchmark_cache/mimic4_ihm_w_pre2/")


Label mortality vocab: {0: 0, 1: 1}
Processing samples and saving to ../benchmark_cache/mimic4_ihm_w_pre2...
Create an account on https://lightning.ai/ to optimize your data faster using multiple nodes and large machines.
Setting multiprocessing start_method to fork. Tip: Libraries relying on lock can hang with `fork`. To use `spawn` in notebooks, move your code to files and import it within the notebook.
Storing the files under /home/johnwu3/projects/PyHealth_Branch_Testing/PyHealth/benchmark_cache/mimic4_ihm_w_pre2
Setup started with fast_dev_run=False.
Setup finished in 0.021 seconds. Found 1824 items to process.
Starting 4 workers with 1824 items. The progress bar is only updated when a worker finishes.
Workers are ready ! Starting data processing...


Progress:   0%|          | 0/1824 [00:00<?, ?it/s]

Rank 0 inferred the following `['str', 'str', 'tensor', 'no_header_tensor:1']` data format.
Rank 1 inferred the following `['str', 'str', 'tensor', 'no_header_tensor:1']` data format.
Rank 2 inferred the following `['str', 'str', 'tensor', 'no_header_tensor:1']` data format.
Rank 3 inferred the following `['str', 'str', 'tensor', 'no_header_tensor:1']` data format.
Worker 0 is terminating.
Worker 1 is terminating.
Worker 2 is terminating.
Worker 0 is done.
Worker 3 is terminating.
Worker 1 is done.
Worker 2 is done.
Worker 3 is done.
Workers are finished.
Finished data processing!
Cached processed samples to ../benchmark_cache/mimic4_ihm_w_pre2


In [4]:
len(samples)

912

In [2]:
dataset.get_patient(dataset.unique_patient_ids[0]).get_events()

Found 1000 unique patient IDs


[Event(event_type='patients', timestamp=datetime.datetime(2025, 12, 18, 21, 8, 17, 423997), attr_dict={'gender': 'F', 'anchor_age': '56', 'anchor_year': '2154', 'anchor_year_group': '2011 - 2013', 'dod': None}),
 Event(event_type='labevents', timestamp=datetime.datetime(2155, 11, 24, 7, 59), attr_dict={'itemid': '50887', 'label': 'Blue Top Hold', 'fluid': 'Blood', 'category': 'Chemistry', 'value': 'HOLD.  DISCARD GREATER THAN 24 HRS OLD.', 'valuenum': None, 'valueuom': None, 'flag': None, 'storetime': None}),
 Event(event_type='labevents', timestamp=datetime.datetime(2155, 11, 24, 7, 59), attr_dict={'itemid': '50933', 'label': 'Green Top Hold (plasma)', 'fluid': 'Blood', 'category': 'Chemistry', 'value': 'HOLD.  DISCARD GREATER THAN 4 HOURS OLD.', 'valuenum': None, 'valueuom': None, 'flag': None, 'storetime': None}),
 Event(event_type='labevents', timestamp=datetime.datetime(2155, 11, 24, 7, 59), attr_dict={'itemid': '51301', 'label': 'White Blood Cells', 'fluid': 'Blood', 'category': 

In [3]:
samples[0]

{'patient_id': '15853169',
 'admission_id': '23443189',
 'labs': tensor([[  0.0000,   0.0000, 139.0000,   0.0000,   0.0000,   0.0000,   4.4000,
            0.0000,   0.0000,   0.0000, 103.0000,   0.0000,   0.0000,   0.0000,
            0.0000,   0.0000,  83.0000,   0.0000,   0.0000,   0.0000,   2.2000,
           16.0000,   0.0000,   0.0000,   0.0000,   0.0000,   4.7000]]),
 'mortality': tensor([0.])}

In [4]:
from pyhealth.datasets import get_dataloader
train_dataloader = get_dataloader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = get_dataloader(val_dataset, batch_size=32, shuffle=False)
test_dataloader = get_dataloader(test_dataset, batch_size=32, shuffle=False)

from pyhealth.models import RNN

model = RNN(
    dataset=samples,
)

from pyhealth.trainer import Trainer

trainer = Trainer(model=model, metrics=["roc_auc"])
print(trainer.evaluate(test_dataloader))

trainer.train(
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    epochs=10,
    monitor="roc_auc",  # Monitor roc_auc specifically
    optimizer_params={"lr": 1e-4}  # Using learning rate of 1e-4
)


/home/johnwu3/projects/PyHealth_Branch_Testing/PyHealth/pyhealth/sampler/sage_sampler.py:3: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


RNN(
  (embedding_model): EmbeddingModel(embedding_layers=ModuleDict(
    (labs): Linear(in_features=27, out_features=128, bias=True)
  ))
  (rnn): ModuleDict(
    (labs): RNNLayer(
      (dropout_layer): Dropout(p=0.5, inplace=False)
      (rnn): GRU(128, 128, batch_first=True)
    )
  )
  (fc): Linear(in_features=128, out_features=1, bias=True)
)
Metrics: ['roc_auc']
Device: cuda



Evaluation: 100%|██████████| 6/6 [00:00<00:00, 20.72it/s]

{'roc_auc': 0.8935969868173259, 'loss': 0.6720269918441772}
Training:
Batch size: 32
Optimizer: <class 'torch.optim.adam.Adam'>
Optimizer params: {'lr': 0.0001}
Weight decay: 0.0
Max grad norm: None
Val dataloader: <torch.utils.data.dataloader.DataLoader object at 0x7f8dd01024b0>
Monitor: roc_auc
Monitor criterion: max
Epochs: 10
Patience: None



Epoch 0 / 10:   0%|          | 0/20 [00:00<?, ?it/s]

...

--- Eval epoch-8, step-180 ---
roc_auc: 0.8116
loss: 0.1714



Epoch 9 / 10:   0%|          | 0/20 [00:00<?, ?it/s]

--- Train epoch-9, step-200 ---
loss: 0.1135


Evaluation: 100%|██████████| 3/3 [00:00<00:00, 55.26it/s]

--- Eval epoch-9, step-200 ---
roc_auc: 0.8093
loss: 0.1719
Loaded best model
